In [1]:
import pandas as pd
import pickle
import bz2
import importlib
import re
from collections import Counter
import copy
if __name__ == '__main__':
    import utils
else:
    utils = importlib.import_module('src.utils')

In [9]:
mpa_db = pickle.load(bz2.open('/shares/CIBIO-Storage/CM/scratch/users/francesco.beghini/hg/chocophlan/export/metaphlan2/mpa_v25_CHOCOPhlAn_0.21/mpa_v25_CHOCOPhlAn_0.21.orig.nomerged.pkl'))
all_mpa_gca = [x.split('|')[7][3:] for x in mpa_db['taxonomy'].keys()]
count_marker_per_clade = Counter(v['taxon'] for v in mpa_db['markers'].values())
sgb_release = pd.read_csv('/shares/CIBIO-Storage/CM/scratch/users/francesco.beghini/hg/sgbrepo/releases/SGB.Jan19.txt.bz2', sep='\t')
r = re.compile(r'(.*(C|c)andidat(e|us)_.*)|(.*_sp(_.*|$))|((.*_|^)(b|B)acterium(_.*|$))|(.*(eury|)archaeo(n_|te).*)|(.*(endo|)symbiont.*)')

In [10]:
gca2taxonomy = pd.read_csv('/shares/CIBIO-Storage/CM/scratch/users/francesco.beghini/hg/chocophlan/export/gca2taxonomy_201804.txt', sep='\t')
gca2taxonomy = dict(zip(gca2taxonomy.GCA_accession, gca2taxonomy.taxstr))
bad_genomes = {k:v for k,v in gca2taxonomy.items() if r.match(v) and k in all_mpa_gca}

In [11]:
gca2sgb = dict((y, (x[0],x[7])) for _, x in sgb_release.iterrows() for y in x[4].split(',') if y in bad_genomes)
merged = {}
keep = {}
for genome, (sgb, taxonomy) in gca2sgb.items():
    nrefgenomes = sgb_release.loc[sgb_release['SGB ID'] == sgb, '# Reference genomes'].item()
    #If genome is unique in the SGB, keep it
    if nrefgenomes == 1:
        keep.setdefault(bad_genomes[genome],[]).append(genome)
    #If there are multiple refences, keep only one and the others list in a list of merged genomes
    else:
        merged.setdefault(sgb,[]).append((genome,bad_genomes[genome]))

In [12]:
new_species_merged = {}
for sgb, gca_tax in merged.items():
    merged_into='|'.join(sgb_release.loc[sgb_release['SGB ID'] == sgb, 'Assigned taxonomy'].item().split('|')[:7])
    if merged_into not in count_marker_per_clade:
        species_in_sgb = set(taxstr for (gca, taxstr) in gca_tax)
        merged_into=max([(s,count_marker_per_clade[s]) for s in species_in_sgb],key = lambda x:x[1])[0]
    new_species_merged.setdefault(merged_into,[]).extend(gca_tax)

In [6]:
with open('/shares/CIBIO-Storage/CM/scratch/users/francesco.beghini/hg/chocophlan/export/metaphlan2/merged_species_spp.tsv', 'wt') as fout:
    fout.write('GCA\told_taxonomy\tmerged_into\n')
    fout.write('\n'.join('{}\t{}\t{}'.format(gca, old_t, new_t) for new_t, gca_tax in new_species_merged.items() for gca, old_t in gca_tax))
    fout.write('\n'.join('{}\t{}\t{}'.format(gca, taxa, taxa) for taxa, gca_tax in keep.items() for gca in gca_tax))

In [13]:
taxa2markers = {}
[taxa2markers.setdefault(v['taxon'],[]).append(k) for k,v in mpa_db['markers'].items()]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
for new_t, gca_tax in new_species_merged.items():
    for gca, old_t in gca_tax:
        if new_t != old_t and old_t in taxa2markers:
            [mpa_db['markers'].pop(marker) for marker in taxa2markers[old_t] if marker in mpa_db['markers']]
            mpa_db['taxonomy'].pop('{}|t__{}'.format(old_t,gca))

In [18]:
all_gca = [x.split('|')[-1].split('__')[1] for x in mpa_db['taxonomy'].keys()]
for x in mpa_db['markers']:
    mpa_db['markers'][x]['ext'] = [y for y in set(mpa_db['markers'][x]['ext']) if y in all_gca]

In [19]:
with bz2.BZ2File('/shares/CIBIO-Storage/CM/scratch/users/francesco.beghini/hg/chocophlan/export/metaphlan2/mpa_v25_CHOCOPhlAn_0.21/mpa_v25_CHOCOPhlAn_0.21.pkl', 'w') as outfile:
    pickle.dump(mpa_db, outfile, pickle.HIGHEST_PROTOCOL)